In [1]:
from __future__ import absolute_import
from datetime import datetime, timedelta, date
import hashlib
import requests
import os
import sys
from os.path import *
import urllib
import xlrd
sys.path.insert(0, '../')
try:
    from bs4 import (
        BeautifulSoup, Tag, Comment, ProcessingInstruction, NavigableString,
        Declaration, Doctype)
    _DECLARATION_OR_DOCTYPE = (Declaration, Doctype)
except ImportError:
    from BeautifulSoup import (
        BeautifulSoup, Tag, Comment, ProcessingInstruction, NavigableString,
        Declaration)
    _DECLARATION_OR_DOCTYPE = Declaration
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as cond
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from pyvirtualdisplay.display import Display
from selenium.webdriver.common.by import By

In [2]:
from selenium.webdriver.support.ui import Select

In [3]:
import time #using this library to pause or "sleep" program

In [4]:
driver=webdriver.Firefox(executable_path=f'C:\Drivers\FirefoxDriver\geckodriver.exe')

C:\Users\tatib\AppData\Local\Temp\ipykernel_17664\1708203200.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Firefox(executable_path=f'C:\Drivers\FirefoxDriver\geckodriver.exe')


In [7]:
driver.implicitly_wait(100) #seconds

In [6]:
driver.maximize_window()

In [8]:
driver.get('https://gisis.imo.org/Public/SHIPS/ShipSearch.aspx')

In [9]:
#select 'Public Users' from drop down menu
user_authority= Select(driver.find_element(By.ID, "ctl00_cpMain_ddlAuthorityType"))
user_authority.select_by_visible_text("Public Users")

In [10]:
#enter username
driver.find_element(By.ID, "ctl00_cpMain_txtUsername").send_keys("test_account")

In [11]:
#click the 'next' button
driver.find_element(By.ID, "ctl00_cpMain_btnNext").click()

In [12]:
#enter password 
driver.find_element(By.ID, "ctl00_cpMain_txtPassword").send_keys("RC5@862z8CWE$wS")

In [13]:
#click log in button 
driver.find_element(By.ID, "ctl00_cpMain_btnLogin").click()

In [14]:
#select Ship Type (detailed) from drop down menu
ship_search= Select(driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_shipWhereBuilder_fields"))
ship_search.select_by_visible_text("Ship type (detailed)")

In [15]:
#select first type of ship, press add, and then change "and" to "or". Then proceed to add the rest of the ships, pressing add after 
#each drop down selection. 
ship_type=Select(driver.find_element(By.ID, "GISIS_SHIPS_ShipSelector_ddlLevel5ShipTypes"))
ship_type.select_by_visible_text("FPSO, Oil (Other Offshore)")

In [16]:
#press the "add" button
driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_shipWhereBuilder_entryButtons").click()

In [17]:
#Change "and" to "or" in the drop down menu.
#first find the drop down menu.
ship_search_and_or=Select(driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_shipWhereBuilder_andOr"))
#now select "or"
ship_search_and_or.select_by_visible_text("or")

In [18]:
#select second ship type
ship_type.select_by_visible_text("FSO, Gas (Other Offshore)")
#press add
driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_shipWhereBuilder_entryButtons").click()

In [19]:
#select third ship type 
ship_type.select_by_visible_text("FSO, Oil (Other Offshore)")
#press add
driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_shipWhereBuilder_entryButtons").click()

In [20]:
#now press "start search" button
driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_btnSearch").click()

In [21]:
#find max page number. 
pages=driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_gvShip_ctl01_pnlPaging").text
max_page_split=pages.split("of ")
max_page=int(max_page_split[-1])+1 #add 1 to number of pages; outer loop control variable
print(max_page)

40


In [22]:
#count number of rows on main table.
#test to see that the following code works - if we're on the first page, should get 18 as our result. if we're 
#on the last page, we may get less. 
rows=driver.find_elements(By.XPATH, "//table[@class='gridviewer_grid']/tbody/tr")
max_rows=len(rows)+1
print(max_rows)

18


In [25]:
#nested for loops to traverse through main table, press button to get to 3 tables, go three each of three tables, 
#press back button, and repeat.
max_rows=18 #always going to be 18 except possibly the last page, assuming there are 15 rows per full page. 
for i in range(1, max_page, 1): #(initialize at 1, upper bound is 4, which means that we are only traversing 3 pages, and increment by 1 page number.    
    if i==(max_page-1): #if we're on the last page, we need to count our rows, because there may be less than 15.
        rows=driver.find_elements(By.XPATH, "//table[@class='gridviewer_grid']/tbody/tr")
        max_rows=len(rows)+1    
    for i in range(3, max_rows, 1): #(initialization, stop, increment) - we want to increment tr[i], with i initialized to 3.
        ship_name_cell=driver.find_element(By.XPATH, "//table[@id='ctl00_bodyPlaceHolder_ShipSelector_gvShip']/tbody/tr["+str(i)+"]/td[1]").click()
        heading = driver.find_element(By.TAG_NAME, "h1")
        print(heading.text)
        subheading=driver.find_element(By.CLASS_NAME, "desc")
        print(subheading.text+"\n------------------------------")

        print("Table 1"+"\n-------")
        #table 1
        for i in range(1, 8, 1): 
            row_label=driver.find_element(By.XPATH, "//table[@id='wrapper']/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr["+str(i)+"]/td[1]").text
            column_info=driver.find_element(By.XPATH, "//table[@id='wrapper']/tbody/tr/td/table/tbody/tr[2]/td/div/table[1]/tbody/tr["+str(i)+"]/td[2]").text
            print(row_label, column_info)

        print("\nTable 2"+"\n-------")
        #table 2
        for i in range(1, 4, 1): #(initialization, stop, increment)
            row_label=driver.find_element(By.XPATH, "//table[@id='wrapper']/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr["+str(i)+"]/td[1]").text
            column_info=driver.find_element(By.XPATH, "//table[@id='wrapper']/tbody/tr/td/table/tbody/tr[2]/td/div/table[2]/tbody/tr["+str(i)+"]/td[2]").text
            print(row_label, column_info)

        print("\nTable 3"+"\n-------")
        #table 3
        #no for loop here since there's only 1 row.
        row_label=driver.find_element(By.XPATH, "//table[@id='wrapper']/tbody/tr/td/table/tbody/tr[2]/td/div/table[3]/tbody/tr[1]/td[1]").text
        column_info=driver.find_element(By.XPATH, "//table[@id='wrapper']/tbody/tr/td/table/tbody/tr[2]/td/div/table[3]/tbody/tr[1]/td[2]").text
        print(row_label, column_info)
        print()

        #click 'back to results' button after you are done getting info from all 3 tables.
        driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_btnBack").click()
        time.sleep(3)

    next_page_button=driver.find_element(By.ID, "ctl00_bodyPlaceHolder_ShipSelector_gvShip_ctl01_btnNext")      
    if next_page_button.is_enabled():
        next_page_button.click()
        time.sleep(3)
    else: 
        print("Done.")

Ship Particulars / IMO 5033997
LES SYLPHIDES
------------------------------
Table 1
-------
Name: LES SYLPHIDES
IMO Number: IMO 5033997
Flag: Spain
Call sign: EDKM
MMSI: 
Ship UN Sanction: Not on list
Owning/operating entity under UN Sanction: Not on list

Table 2
-------
Type: FSO, Oil
Converted from: Oil Tanker-197200
Date of build: 1961-10
Gross tonnage: 20,867

Table 3
-------
Registered owner: LES SYLPHIDES (1040091)

Ship Particulars / IMO 5136804
GROZNYY
------------------------------
Table 1
-------
Name: GROZNYY
IMO Number: IMO 5136804
Flag: Saint Vincent and the Grenadines
Call sign: J8DN8
MMSI: 
Ship UN Sanction: Not on list
Owning/operating entity under UN Sanction: 

Table 2
-------
Type: FSO, Oil
Date of build: 1953-12
Gross tonnage: 7,653

Table 3
-------
Registered owner: QUICK TRADE LTD (1404110)

Ship Particulars / IMO 5207990
LIEPAYA
------------------------------
Table 1
-------
Name: LIEPAYA
IMO Number: IMO 5207990
Flag: Ukraine
Call sign: UUFI
MMSI: 
Ship UN Sanct